LLM大语言模型Generate/Inference生成或者说推理时，有很多的参数和解码策略，比如OpenAI在提供GPT系列的模型时，就提供了很多的参数[1]，那这些参数的原理以及代码上怎么实现的呢？本文将尽力进行一一的解释。

# 原始生成

假如都没有使用这些参数和策略做后处理，模型是怎么生成的呢？以llama模型举例（其他生成式模型是一样）：

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "llama-2-7b-hf" # 用你的模型的地址
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

text = "say"
inputs = tokenizer(text, return_tensors="pt")
print(f"inputs:{inputs}")

# 结果
inputs:{'input_ids': tensor([[   1, 1827]]), 'attention_mask': tensor([[1, 1]])}

我们输入的模型的就一个词：say（该词分词后就是1个token，在词表中的位置是1827），然后给到模型预测下一个token，如果我们不做任何参数控制，就是直接走模型的forward：

In [ ]:

logits = model.forward(input_ids)
print("Logits Shape:", logits.logits.shape)
print(f"logits:{logits.logits}")

# 结果
Logits Shape: torch.Size([1, 2, 32000])
logits:tensor([[[-12.9696,  -7.4491,  -0.4354,  ...,  -6.8250,  -8.0804,  -7.5782],
         [-11.3775, -10.1338,  -2.3563,  ...,  -6.7709,  -6.5252,  -8.9753]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>)

logits就是模型的最后输出，是一个张量（tensor），它的维度是[batch_size, sequence_length, vocab_size]，在我们这里，batch_size是1，sequence_length是2，因为我们输入是一个 "say"，而模型在生成下一个token，因此生成的序列长度为2（"say" + 1个生成的token），vocab_size是词典的大小，llama是32000，也就是说咱们接下来要在第2个sequence里找到在32000词表中哪个token的概率最大：

In [ ]:
# 在最后一个维度上（32000）进行求最大值操作，返回具有最高分数的词汇索引
next_token = torch.argmax(logits.logits, dim=-1).reshape(-1)[1]
print(f"next_token:{next_token}")

# 结果
next_token:tensor([22172], device='cuda:0')

在最后一个维度上（32000）进行求最大值操作，并返回具有最高分数的词汇索引，在词表中的位置是22172，接下来就是解码该token：

In [ ]:
next_word = tokenizer.decode(next_token)
print(f"next_word:{next_word}")

# 结果
next_word:hello

这就将next_word预测了出来，后面的流程就是将“hello”加到“say”后面变成“say hello”，迭代上述流程直到生成eos_token（终止词），整个预测也就完成了。上述就是不加任何参数和后处理的生成式模型的generate/inference全过程，这个过程也叫做greedy decoding贪心解码策略，下文会介绍。

# 常见参数（Huggingface中的常用参数[2]）

# 1.temperature

该参数用于控制生成文本的随机性和多样性，其实是调整了模型输出的logits概率分布，实现原理很简单，我们举一个简单的例子，假设我们有一个大小为[1, 4]的logits张量，在上述原始生成例子中其实是[1, 32000]，然后将logits输入到softmax函数中，分别计算没有temperature，以及temperature为0.5和temperature为2的情况下的概率分布：

In [ ]:
import torch
logits = torch.tensor([[0.9, 1.2, -1.0, 0.1]])
# 无temperature
probs = torch.softmax(logits, dim=-1)
# temperature low 0.5
probs_low = torch.softmax(logits / 0.5, dim=-1)
# temperature high 2
probs_hige = torch.softmax(logits / 2, dim=-1)
# 结果
probs: tensor([[0.2559, 0.5154, 0.0571, 0.1716]])
probs_low: tensor([[0.1800, 0.7301, 0.0090, 0.0809]])
probs_high: tensor([[0.2695, 0.3825, 0.1273, 0.2207]])

从上述结果中可以看到，当temperature较高时，会更平均地分配概率给各个token，这导致生成的文本更具随机性和多样性；temperature较低接近0时，会倾向于选择概率最高的token，从而使生成的文本更加确定和集中。注：temperature=1时表示不使用此方式。

# 2.top_p

top-p也是一个用于控制生成文本多样性的参数，也被称为"nucleus sampling"。这个参数的全名是"top probability"，通常用一个介于 0 到 1 之间的值来表示生成下一个token时，在概率分布中选择的最高概率的累积阈值，我们看一下是怎么实现的，还以上述的例子为例：

In [ ]:
import torch
# 样例：probs: tensor([[0.2559, 0.5154, 0.0571, 0.1716]])
probs = torch.tensor([[0.2559, 0.5154, 0.0571, 0.1716]])
# 第一步进行排序
probs_sort, probs_idx = torch.sort(probs, dim=-1, descending=True)
# 结果
probs_sort: tensor([[0.5154, 0.2559, 0.1716, 0.0571]])
probs_idx: tensor([[1, 0, 3, 2]])

# 第二步概率的累积和
probs_sum = torch.cumsum(probs_sort, dim=-1)
# 结果
probs_sum: tensor([[0.5154, 0.7713, 0.9429, 1.0000]])

# 第三步找到第一个大于阈值p的位置，假设p=0.9，并将后面的概率值置为0：
mask = probs_sum - probs_sort > p
probs_sort[mask] = 0.0
# 结果
probs_sort: tensor([[0.5154, 0.2559, 0.1716, 0.0000]])

# 第四步复原原序列
new_probs = probs_sort.scatter(1, probs_idx, probs_sort)
# 结果
new_probs: tensor([[0.2559, 0.5154, 0.0000, 0.1716]])

# 注：在真实实现中一般会把舍弃的概率置为-inf，即
zero_indices = (new_probs == 0)
new_probs[zero_indices] = float('-inf')
# 结果
new_probs: tensor([[0.2559, 0.5154, -inf, 0.1716]])
# 完整代码
def sample_top_p(probs, p):
    probs_sort, probs_idx = torch.sort(probs, dim=-1, descending=True)
    probs_sum = torch.cumsum(probs_sort, dim=-1)
    mask = probs_sum - probs_sort > p
    probs_sort[mask] = 0.0
    new_probs = probs_sort.scatter(1, probs_idx, probs_sort)
    zero_indices = (new_probs == 0)
    new_probs[zero_indices] = float('-inf')
    return new_probs

从上述的实现中可以看到，当top_p较高时比如 0.9，这意味着前 90% 的概率的token会被考虑在抽样中，这样会允许更多的token参与抽样，增加生成文本的多样性；当top_p较低时比如比如 0.1，这意味着只有前 10% 最高概率的token会被考虑在抽样中，这样会限制生成文本的可能性，使生成的文本更加确定和集中。这里可能会有一点疑问，当top-p设置的很小，累加的概率没超过怎么办？一般代码中都会强制至少选出一个token的。注：top_p=1时表示不使用此方式。

# 3.top_k

这个参数比较简单，简单来说就是用于在生成下一个token时，限制模型只能考虑前k个概率最高的token，这个策略可以降低模型生成无意义或重复的输出的概率，同时提高模型的生成速度和效率。实现如下：

In [ ]:
import torch
filter_value = -float("Inf")
top_k = 2
probs = torch.tensor([[0.2559, 0.5154, 0.0571, 0.1716]])
indices_to_remove = probs < torch.topk(probs, top_k)[0][..., -1, None]
new_probs = probs.masked_fill(indices_to_remove, filter_value)
print("new_probs:", new_probs)
# 结果
new_probs: tensor([[0.2559, 0.5154,   -inf,   -inf]])

# 4.repetition_penalty

这个重复惩罚参数也比较容易理解，通过修改生成文本时的概率分布来实现的， repetition_penalty的目标是在这个概率分布中对先前生成过的token，又重复的生成了该token进行惩罚（降低概率），以减少生成文本中的重复性，简单实现如下：

In [ ]:
import numpy as np
def apply_repetition_penalty(probs, repetition_penalty, prev_tokens):
    adjusted_probs = np.copy(probs)
    for token in set(prev_tokens):
        adjusted_probs[token] *= repetition_penalty
    adjusted_probs /= np.sum(adjusted_probs)  
    return adjusted_probs
# 示例概率分布，索引对应不同词语
original_probs = np.array([0.3, 0.1, 0.3, 0.1, 0.2])
# 示例先前生成的词语
previous_tokens = [2, 4, 2]
# 重复惩罚系数
repetition_penalty = 0.8
# 应用重复惩罚，得到调整后的概率分布
adjusted_probs = apply_repetition_penalty(original_probs, repetition_penalty, previous_tokens)

print("原始概率分布：", original_probs)
print("调整后的概率分布：", adjusted_probs)

# 结果
原始概率分布： [0.3 0.1 0.3 0.1 0.2]
调整后的概率分布： [0.33333333 0.11111111 0.26666667 0.11111111 0.17777778]

上述结果很明显看的出来，出现过的的token概率变低了，未出现过的token的概率变高了。注：repetition_penalty=1时表示不进行惩罚。

# 5.no_repeat_ngram_size

这个参数，当设为大于0的整数时，生成的文本中不会出现指定大小的重复n-gram（n个连续的token），可以使生成的文本更加多样化，避免出现重复的短语或句子结构。实现原理和上述repetition_penalty的是大致差不多的，只不过这里是n个连续的token。注默认不设置

# 6.do_sample

这个参数是对模型计算出来的概率要不要进行多项式采样，多项式采样（Multinomial Sampling）是一种用于从一个具有多个可能结果的离散概率分布中进行随机抽样的方法，多项式采样的步骤如下：

首先，根据概率分布对应的概率，为每个可能结果分配一个抽样概率。这些抽样概率之和必须为1。

然后，在进行一次抽样时，会根据这些抽样概率来选择一个结果。具体地，会生成一个随机数，然后根据抽样概率选择结果。抽样概率越高的结果，被选中的概率也就越大。

最终，被选中的结果就是这次抽样的输出。

在多项式采样中，概率高的结果更有可能被选中，但不同于确定性的选择，每个结果仍然有一定的概率被选中。这使得模型在生成文本时具有一定的随机性，但又受到概率的控制，以便生成更加多样且符合概率分布的文本。实现如下：

In [ ]:
import torch
probs = torch.tensor([[0.2559, 0.5154, 0.0571, 0.1716]])
next_token = torch.multinomial(probs, num_samples=1)
print("next_token:", next_token)
# 结果
next_token: tensor([[1]])

这个do_sample参数通过多样式采样会有一定的随机性，这种随机性导致了生成的文本更加多样化，因为模型有机会选择概率较低但仍然可能的词，这种方法可以产生丰富、有趣、创新的文本，但可能会牺牲一些文本的准确性。注do_sample=False，不进行采样。在Huggingface中，do_sample这个参数有更高的含义即做不做多样化采样，当do_sample=False时，temperature，top_k，top_p这些参数是不能够被设置的，只有do_sample=True时才能够被设置。比如：

In [ ]:
UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.

# 7.num_beams

num_beams参数是用于束搜索（beam search）算法的，其用途是控制生成的多个候选句子的数量，该参数控制的是每个生成步要保留的生成结果的数量，用于在生成过程中增加多样性或生成多个可能的结果。主要步骤如下：

在每个生成步，对于前一个生成中的所有生成结果，分别基于概率保留前 k 个最可能的结果（k 即 num_beams 参数的值）。

将所有扩展后的生成结果，按照其对应的概率分数重新计算分数并进行排序，并保留前 k 个最可能的结果。

如果已经生成了结束符，则将其对应的结果保留下来。

重复上述过程直到生成所有的结果或达到最大长度。

以下是简单代码实现，结合代码会更容易理解：

首先定义一个BeamSearchNode类

In [ ]:
class BeamSearchNode:
    def __init__(self, sequence, score):
        self.sequence = sequence  # 生成的序列
        self.score = score  # 分数（概率）

In [ ]:
# 示例：下一个token的概率函数，简单使用固定概率
def simple_next_word_probs(sequence):
    if sequence[-1] == "<end>":
        return {}
    return {"apple": 0.3, "like": 0.35, "peach": 0.2, "banana": 0.15}

下面是beam_search算法的非常简单的实现，工程上实现的话参考Huggingface的官方实现[2]，

In [ ]:
def beam_search(initial_sequence, next_word_probs_func, num_beams, max_sequence_length):
    # 初始化初始节点，且分数为1
    initial_node = BeamSearchNode(sequence=initial_sequence, score=1.0)
    candidates = [initial_node]

    final_candidates = []  # 最终的候选序列
    # 只要候选节点列表不为空，且 final_candidates 中的候选节点数量还没有达到指定的束宽度，就继续进行搜索
    while candidates and len(final_candidates) < num_beams:
        # 候选节点排序
        candidates.sort(key=lambda x: -x.score)
        current_node = candidates.pop(0)
        # 当节点序列末尾生成结束符号（如"<end>"），或者当生成的序列长度达到最大限制时终止节点的扩展
        if current_node.sequence[-1] == "<end>" or len(current_node.sequence) >= max_sequence_length:
            final_candidates.append(current_node)
        else:
            # 获取下一个token的概率，我们的例子返回的是固定的概率
            next_words_probs = next_word_probs_func(current_node.sequence) 
            # 生成新的候选序列，并计算分数           
            for next_word, next_word_prob in next_words_probs.items():
                new_sequence = current_node.sequence + [next_word]
                new_score = current_node.score * next_word_prob
                new_node = BeamSearchNode(sequence=new_sequence, score=new_score)
                candidates.append(new_node)

    return [candidate.sequence for candidate in final_candidates]

开始使用：

In [ ]:
initial_sequence = ["<start>", "I"]
num_beams = 3
max_sequence_length = 3
result = beam_search(initial_sequence, simple_next_word_probs, num_beams, max_sequence_length)

for idx, sequence in enumerate(result):
    print(f"Sentence {idx + 1}: {' '.join(sequence)}")

# 结果，符合我们的概率分布
Sentence 1: <start> I like
Sentence 2: <start> I apple
Sentence 3: <start> I peach

# 当长度为4时呢？
Sentence 1: <start> I like like
Sentence 2: <start> I like apple
Sentence 3: <start> I apple like

# 为什么结果变了呢？我们来看一下最终每个序列的分数
current_node: ['<start>', 'I', 'like', 'like']
current_score: 0.12249999999999998
current_node: ['<start>', 'I', 'like', 'apple']
current_score: 0.105
current_node: ['<start>', 'I', 'apple', 'like']
current_score: 0.105

# 再看一下其他序列的分数，apple的
new_node: ['<start>', 'I', 'apple', 'apple']
new_score: 0.09
new_node: ['<start>', 'I', 'apple', 'like']
new_score: 0.105
new_node: ['<start>', 'I', 'apple', 'peach']
new_score: 0.06
new_node: ['<start>', 'I', 'apple', 'banana']
new_score: 0.045

# 再看一下其他序列的分数，peach的
new_node: ['<start>', 'I', 'peach', 'apple']
new_score: 0.06
new_node: ['<start>', 'I', 'peach', 'like']
new_score: 0.06999999999999999
new_node: ['<start>', 'I', 'peach', 'peach']
new_score: 0.04000000000000001
new_node: ['<start>', 'I', 'peach', 'banana']
new_score: 0.03

上述就是beam search的简单代码实现，有优点也有相应的缺点

优点：

生成多样性： 通过增加num_beams束宽，束搜索可以保留更多的候选序列，从而生成更多样化的结果。

找到较优解： 增加num_beams束宽有助于保留更多可能的候选序列，从而更有可能找到更优的解码结果，这在生成任务中有助于避免陷入局部最优解

控制输出数量： 通过调整num_beams束宽，可以精确控制生成的候选序列数量，从而平衡生成结果的多样性和数量。

缺点：

计算复杂度： 随着num_beams束宽的增加，计算复杂度呈指数级增长，较大的束宽会导致解码过程变得更加耗时，尤其是在资源有限的设备上。

忽略概率较低的序列： 增加num_beams束宽可能会导致一些低概率的候选序列被忽略，因为搜索过程倾向于集中在概率较高的路径上，从而可能错过一些潜在的优质解。

缺乏多样性： 尽管增加num_beams束宽可以增加生成结果的多样性，但束搜索仍然可能导致生成的结果过于相似，因为它倾向于选择概率较高的路径。

# 8.num_beam_groups

Huggingface中的生成参数中也是有该参数的，这个参数背后其实是一种beam search算法的改进，叫做Diverse Beam Search (DBS)[3]，上述我们已经讨论了beam search生成的结果还是会过于相似的，这个DBS做了一些改进，核心就是分组机制，举个例子来说如果我的num_beams=2，num_beam_groups=2，那就是说分成2个组，每个组里的beam可以相似，但组和组之间要有足够的多样性，引入了多样性分数，具体实现细节可以看一下论文，不过以下一张图就容易理解了：

# 9.diversity_penalty

这个多样性惩罚参数只有在启用了“num_beam_groups”（组束搜索）时才有效，在这些组之间应用多样性惩罚，以确保每个组生成的内容尽可能不同。

# 10.length_penalty

这个长度惩罚参数也是用于束搜索过程中，在束搜索的生成中，候选序列的得分通过对数似然估计计算得到，即得分是负对数似然。length_penalty的作用是将生成序列的长度应用于得分的分母，从而影响候选序列的得分，当length_penalty > 1.0时，较长的序列得到更大的惩罚，鼓励生成较短的序列；当length_penalty< 1.0时，较短的序列得到更大的惩罚，鼓励生成较长的序列，默认为1，不受惩罚。

# 11.use_cache

该参数如何设置为True时，则模型会利用之前计算得到的注意力权重（key/values attentions）的缓存，这些注意力权重是在模型生成文本的过程中，根据输入上下文和已生成部分文本，计算出来的，当下一个token需要被生成时，模型可以通过缓存的注意力权重来重用之前计算的信息，而不需要重新计算一次，有效地跳过重复计算的步骤，从而减少计算负担，提高生成速度和效率。

# 其他简单、少见参数

接下来对比较简单和少见的参数做一下简单阐述

# 1.num_return_sequences

该参数是模型返回不同的文本序列的数量，要和beam search中的num_beams一致，在贪心解码策略中（下述会讲到），num_return_sequences只能为1，默认也为1。

# 2.max_length

生成的token的最大长度。它是输入prompt的长度加上max_new_tokens的值。如果同时设置了max_new_tokens，则会覆盖此参数，默认为20。

# 3.max_new_tokens

生成的最大token的数量，不考虑输入prompt中的token数，默认无设置

# 4.min_length

生成的token的最小长度。它是输入prompt的长度加上min_new_tokens的值。如果同时设置了min_new_tokens，则会覆盖此参数，默认为0。

# 5.min_new_tokens

生成的最小token的数量，不考虑输入prompt中的token数，默认无设置

# 6.early_stopping

控制基于束搜索（beam search）等方法的停止条件，接受以下值：

True：生成会在出现num_beams个完整候选项时停止。

False：应用启发式方法，当很不可能找到更好的候选项时停止生成。

never：只有当不能找到更好的候选项时，束搜索过程才会停止（经典的束搜索算法）。

默认为False

# 7.bad_words_ids

包含词汇id的列表，这个参数用于指定不允许在生成文本中出现的词汇,如果生成的文本包含任何在这个列表中的词汇，它们将被被替换或排除在最终生成的文本之外。

# 8.force_words_ids

包含词汇id的列表，用于指定必须包含在生成文本中的词汇，如果给定一个列表，生成的文本将包含这些词汇。

# 9.constraints

自定义约束条件，可以指定约束条件，这些约束条件可以是必须出现的关键词、短语、特定术语或其他文本元素，其实和force_words_ids是差不多的意思，在代码实现也是一样的。

[1] https://platform.openai.com/docs/api-reference/chat/create

[2] https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationConfig

[3] https://arxiv.org/abs/1610.02424

[4] https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin

[5] https://arxiv.org/abs/2202.06417

[6] https://zhuanlan.zhihu.com/p/599921747


参考文献：https://mp.weixin.qq.com/s/BbWjr8mr3Iu_JLCK0x2rcA